In [1]:
import json, requests, threading
import getpass
from bs4 import BeautifulSoup
from Requester import Requester

### Get Password

In [2]:
password = getpass.getpass(prompt="Enter Password Plis: ")

Enter Password Plis: ········


### Create Session

In [3]:
session = requests.Session()
response = session.post("https://ayamel.byu.edu/auth/password", {"username":"admin", "password":password}, allow_redirects=False)
response = session.get("https://ayamel.byu.edu/admin/content")
session.close()

### Extract Data from Content (html)

In [4]:
class Content:
    def __init__(self, resourceId, name, owner, lang, content_type):
        self.resourceId = resourceId
        self.name = name
        self.owner = owner
        self.lang = lang
        self.content_type = content_type

def extract(row):
    fields = row.find_all('td')
    rid = fields[6].text
    name = fields[1].text
    owner = fields[2].text
    lang = fields[3].text
    content_type = fields[7].text
    return Content(rid, name, owner, lang, content_type)

In [5]:
soup = BeautifulSoup(response.text, "html5lib")
rows = soup.find_all('tr')[1:] # get rid of the first row cause some idiots didn't use table headers
content_list = [extract(x) for x in rows]

### Get Resources

In [6]:
from Requester import CONTENT_OBJS
from Requester import RESOURCE_OBJS

In [7]:
NUM_THREADS = 20

In [8]:
def run_request_process():
    pool = []
    for i in range(NUM_THREADS): pool.append(Requester())
    for t in pool: t.start()
    for t in pool: t.join()

In [9]:
CONTENT_OBJS.extend(content_list) #wedontcare

In [10]:
run_request_process()

### Graphing

In [11]:
import pandas
import numpy as np

In [12]:
# mapping function, convert json to list of values
def values(obj):
    uri = ""
    if obj["status"] == "awaiting_content":
        print("Awaiting content: %s" % str(obj))
        return ["", "", "", "", "", "", ""]
    
    files = obj["content"]["files"][0]
    if "streamUri" in files.keys():
        uri = files["streamUri"]
        
    return [obj["id"], obj["title"], obj["type"], obj["dateAdded"],
            obj["dateModified"], uri, obj["content"]["files"][0]["mimeType"]]

headers = ["id", "title", "type", "dateAdded", "dateModified", "streamUri", "mimeType"]
pandas_rows = [ values(o) for o in RESOURCE_OBJS]

Awaiting content: {'dateAdded': '1443643111', 'relations': [], 'dateModified': '1443643111', 'client': {'name': 'BYU Demos', 'id': 'byu_demo'}, 'type': 'video', 'title': 'Test Video3', 'status': 'awaiting_content', 'id': '560c3ee733e57c500b8b4569'}


In [13]:
data = np.array(pandas_rows)

In [14]:
all_data = pandas.DataFrame(data=data, columns=headers)

In [29]:
def getRowsContainingString(tupl):
    return all_data.loc[all_data[tupl[0]].str.contains(tupl[1])][tupl[0]]
                            
getRowsContainingString(("mimeType", "brightcove"))

AttributeError: 'set' object has no attribute 'str'

In [21]:
all_data.to_csv("all_ayamel_resources.csv", sep=",", encoding="utf-8")